# U-NET TRAINING

### Imports

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm 
from torchmetrics import Accuracy

import os

from PIL import Image
import numpy as np

### Dataset class and Dataloader

In [2]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = sorted(os.listdir(image_dir))
        self.mask_filenames = sorted(os.listdir(mask_dir))
        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        image = Image.open(image_path).convert("RGB")  
        mask = Image.open(mask_path).convert("L")  

        if self.image_transform:
            image = self.image_transform(image)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        return image, mask

image_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.875, 1.0)), 
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

mask_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.875, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

train_dataset = SegmentationDataset(
    image_dir="data/seg/train/images", mask_dir="data/seg/train/masks", image_transform=image_transform, mask_transform=mask_transform
)
val_dataset = SegmentationDataset(
    image_dir="data/seg/val/images", mask_dir="data/seg/val/masks", image_transform=image_transform, mask_transform=mask_transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

### U-NET class

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.enc5 = self.conv_block(512, 1024)

        self.dec4 = self.conv_block(1024, 512)
        self.dec3 = self.conv_block(512, 256)
        self.dec2 = self.conv_block(256, 128)
        self.dec1 = self.conv_block(128, 64)
        
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

        self.pool = nn.MaxPool2d(2)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        enc5 = self.enc5(self.pool(enc4))

        dec4 = self.dec4(F.interpolate(enc5, scale_factor=2, mode='bilinear', align_corners=True))
        dec3 = self.dec3(F.interpolate(dec4, scale_factor=2, mode='bilinear', align_corners=True))
        dec2 = self.dec2(F.interpolate(dec3, scale_factor=2, mode='bilinear', align_corners=True))
        dec1 = self.dec1(F.interpolate(dec2, scale_factor=2, mode='bilinear', align_corners=True))

        out = self.final(dec1)

        return out

model = UNet(in_channels=3, out_channels=1)  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


### Load the weights (skip if you haven't train the model yet)

In [4]:
model.load_state_dict(torch.load("model/segmentation_weights.pth", map_location=device))
print("Pretrained weights loaded successfully!")

Pretrained weights loaded successfully!


### Training loop

In [5]:
criterion = nn.BCEWithLogitsLoss()  
optimizer = optim.Adam(model.parameters(), lr=1e-4)

save_path = "model/segmentation_weights.pth"
num_epochs = 54

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", unit="batch") as tepoch:
        for inputs, masks in tepoch:
            inputs, masks = inputs.to(device), masks.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(1), masks.float().squeeze(1))

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            preds = torch.round(torch.sigmoid(outputs))  
            running_corrects += torch.sum(preds == masks).item()
            total += inputs.size(0) * inputs.size(2) * inputs.size(3)

            tepoch.set_postfix(loss=running_loss / (tepoch.n + 1), accuracy=running_corrects / total)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = running_corrects / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    model.eval()
    val_loss = 0.0
    val_corrects = 0
    val_total = 0
    with torch.no_grad():
        with tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", unit="batch") as vepoch:
            for inputs, masks in vepoch:
                inputs, masks = inputs.cuda(), masks.cuda()

                outputs = model(inputs)
                loss = criterion(outputs.squeeze(1), masks.float().squeeze(1))

                val_loss += loss.item()

                preds = torch.round(torch.sigmoid(outputs)) 
                val_corrects += torch.sum(preds == masks).item()
                val_total += inputs.size(0) * inputs.size(2) * inputs.size(3)

                vepoch.set_postfix(loss=val_loss / (vepoch.n + 1), accuracy=val_corrects / val_total)

    val_loss /= len(val_loader)
    val_accuracy = val_corrects / val_total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if (epoch + 1) % 3 == 0:
        torch.save(model.state_dict(), save_path)
        print(f"Model weights saved at epoch {epoch+1}")

Epoch 1/54 - Training: 100%|██████████| 71/71 [20:34<00:00, 17.39s/batch, accuracy=0.887, loss=0.224]


Epoch 1/54, Loss: 0.2237, Accuracy: 0.8872


Epoch 1/54 - Validation: 100%|██████████| 18/18 [04:59<00:00, 16.64s/batch, accuracy=0.891, loss=0.215]


Validation Loss: 0.2153, Validation Accuracy: 0.8909


Epoch 2/54 - Training: 100%|██████████| 71/71 [03:20<00:00,  2.82s/batch, accuracy=0.89, loss=0.215] 


...

Validation Loss: 0.2198, Validation Accuracy: 0.8869


Epoch 54/54 - Training: 100%|██████████| 71/71 [03:17<00:00,  2.78s/batch, accuracy=0.897, loss=0.196]


Epoch 54/54, Loss: 0.1961, Accuracy: 0.8968


Epoch 54/54 - Validation: 100%|██████████| 18/18 [00:43<00:00,  2.43s/batch, accuracy=0.891, loss=0.213]


Validation Loss: 0.2131, Validation Accuracy: 0.8907
Model weights saved at epoch 54
